In [ ]:
# Retrieve the band, model name, RAM capacity, market regions, and the date when the information was added for device(s) associated with a specified code name.
# Open the CSV file and use 'with' to ensure it's properly closed after reading
with open(file_location, "r") as file:
    data = csv.reader(file)
    code_name = input("Enter the code name: ")

    # Create a flag to check if the code name is found
    code_name_found = False

    # Loop through each row in the CSV file
    for row in data:
        if row and row[7] == code_name:
            # If the code name is found, print the details
            value1 = row[41]
            value2 = row[2]
            value3 = row[23]
            value4 = row[44]
            value5 = row[45]

            print("The Code name {} details are:".format(code_name))
            print("The band(WLAN) of the device is:", value1)
            print("The model name is:", value2)
            print("The Ram of the devics is:", value3)
            print("The market region is:", value4)
            print("The date the information was added is:", value5)

            code_name_found = True
            break  # Exit the loop as we found the ID

    if not code_name_found:
        print("No match for the OEM ID details found.")   

In [ ]:
# Assuming you have a DataFrame 'data_file' with 'brand', 'price', and 'price_currency' columns

# Group the data by 'brand' and 'price_currency' and calculate the average price for each combination
brand_avg_prices = data_file.groupby(['brand', 'price_currency'])['price'].mean().round(2).reset_index()

if not brand_avg_prices.empty:
    num_rows = 20
    start_pos = 0
    end_pos = min(start_pos + num_rows, len(brand_avg_prices))

    while True:
        # Display the current set of rows
        print(brand_avg_prices.iloc[start_pos:end_pos])

        if end_pos >= len(brand_avg_prices):
            print("No more rows available.")
            break

        user_input = input("Enter 'next' to retrieve the next 20 rows, or 'quit' to exit: ")
        if user_input == 'next':
            start_pos += num_rows
            end_pos = min(start_pos + num_rows, len(brand_avg_prices))
        elif user_input == 'quit':
            break


In [ ]:
# user interface
def calculate_average_price_for_band(): 
    # Get a list of unique cellular bands in the dataset
    unique_bands = new_updated_data['individual_bands'].unique()

    # Prompt the user to input the band they want to analyze
    selected_band = input("Enter the cellular band you want to analyze: ")

    if selected_band in unique_bands:
        # Filter data for the selected band
        band_data = new_updated_data[new_updated_data['individual_bands'] == selected_band]

        # Get a list of unique currencies for the selected band
        unique_currencies = band_data['price_currency'].unique()

        # Create a dictionary to store average prices for each currency
        average_prices = {}

        # Calculate the average price for each combination of currency and band
        for currency in unique_currencies:
            condition_data = band_data[band_data['price_currency'] == currency]
            average_price = condition_data['price'].mean()
            average_prices[currency] = average_price

        # Print the result for the selected band
        print(f"Average prices for devices within {selected_band}:")
        for currency, average_price in average_prices.items():
            print(f"Currency: {currency}, Average Price: {average_price:.2f}")
    else:
        print("The selected band is not in the dataset.")
calculate_average_price_for_band()

In [ ]:
def get_band_regions():
    new_updated_data = data_file 
    # Split the 'supported_cellular_bands' into individual bands and create a new column
    new_updated_data['individual_bands'] = new_updated_data['supported_cellular_bands'].str.split(',')

    # Explode the list of bands into separate rows
    new_updated_data = new_updated_data.explode('individual_bands')

    # Use groupby to count the occurrences of each band in each region
    band_counts = new_updated_data.groupby(['market_regions', 'individual_bands']).size().reset_index(name='counts')

    # Now, you can find the top 5 regions for each band type
    top_regions_by_band = band_counts.groupby('individual_bands').apply(lambda x: x.nlargest(5, 'counts'))

    # Drop the 'individual_bands' column to avoid conflicts when resetting the index
    top_regions_by_band = top_regions_by_band.drop(columns='individual_bands')

    # Reset the index to avoid the ambiguity issue
    top_regions_by_band = top_regions_by_band.reset_index()

    # The result will be a DataFrame with the top 5 regions for each band type
    for band, data in top_regions_by_band.groupby('individual_bands'):
        print(f"Band: {band}")
        for _, row in data.iterrows():
            print(f"Region: {row['market_regions']} Count: {row['counts']}")
        print()


get_band_regions()


In [ ]:
def get_cheapest_prices_and_recommendations():
    while True:
        try:
            brand_names = ", ".join(data_file['brand'].unique())
            print(f"Available device brand names in this dataset:\n{brand_names}")

            user_brand = input("Enter the brand to find the top 5 cheapest prices (or enter 'exit' to quit): ").strip().lower()

            if user_brand == 'exit':
                break  # Exit the loop if the user wants to quit

            filtered_data = data_file[data_file['brand'].str.lower() == user_brand]

            if not filtered_data.empty:
                top_5_cheapest = filtered_data.nsmallest(5, 'price')[['oem_id', 'price']]
                print(f"Top 5 cheapest prices for brand: {user_brand}")
                for _, row in top_5_cheapest.iterrows():
                    print(f"OEM ID: {row['oem_id']} - Price: {row['price']}")
                
                print()
                print("Now, you can enter the OEM ID of one of these devices to check for recommendations based on the price-performance ratio.")
                user_id_input = input("Enter the OEM ID (or 'quit' to exit): ")

                if user_id_input == 'quit':
                    break  # Exit the loop if the user wants to quit

                selected_device = filtered_data[filtered_data['oem_id'] == user_id_input]

                if not selected_device.empty:
                    brand = selected_device['brand'].values[0]
                    price = selected_device['price'].values[0]
                    performance_score = selected_device['Performance_Score'].values[0]
                    price_performance_ratio = selected_device['Price_Performance_Ratio'].values[0]

                    print()
                    print(f"For this {brand} brand and OEM ID of {user_id_input} with a price of {price}:")
                    print(f"The Performance score of the device is {performance_score:.2f}")
                    print(f"The higher the number (up to {filtered_data['Performance_Score'].max():.2f}) the better the performance of the device.")
                    print(f"The lower the number (down to {filtered_data['Performance_Score'].min():.2f}) the less the performance of the device.")
                    print("---------------------------------------------------------------------------------------------------------------------")
                    print(f"The Price Performance Ratio for the device is {price_performance_ratio:.2f}")
                    print(f"The higher the number (up to {filtered_data['Price_Performance_Ratio'].max():.2f}) the better the performance relative to the price.")
                    print(f"The closer it is to (down to {filtered_data['Price_Performance_Ratio'].min():.2f}), the less the performance of the device for its price.")

                    if price_performance_ratio >= 5:  # Good buy threshold is 5.0
                        print("This is a good buy! It offers great performance for its price.")
                    else:
                        print("Based on the price performance ratio. \nPlease consider other options. It may not provide the best value for your money.")
                else:
                    print(f"No device with OEM ID {user_id_input} found in the dataset.")
            else:
                print(f"No data available for brand: {user_brand}")
        except Exception as e:
            print(f"An error occurred: {e}. Please try again.")

# Call the function to get the cheapest prices and recommendations
get_cheapest_prices_and_recommendations()


In [ ]:
def price_performance_ratio():

    df = data_file

    # Define weightage factors
    weight_cpu = 0.4
    weight_ram = 0.3
    weight_storage = 0.2
    weight_price = 0.1

    # Calculate a performance score based on specifications
    df['Performance_Score'] = (weight_cpu * df['cpu_clock']) + \
                               (weight_ram * df['ram_capacity']) + \
                               (weight_storage * df['non_volatile_memory_capacity'])

    # Calculate price-performance ratio
    df['Price_Performance_Ratio'] = df['Performance_Score'] / df['price']

    # Normalize the price-performance ratios
    df['Normalized_Price_Performance'] = (df['Price_Performance_Ratio'] - df['Price_Performance_Ratio'].min()) / \
                                        (df['Price_Performance_Ratio'].max() - df['Price_Performance_Ratio'].min())

    # Sort the phones by their normalized price-performance ratios
    df = df.sort_values(by='Normalized_Price_Performance', ascending=False)

    # Print the recommendations
    print("Device recommendations based on Price-Performance Ratio:")


    # print statement for the customer to better understand the results
    # Ask the user for input
    user_id_input = input("Enter the oem_id of the device: ")

    # Find and display information about the specified device
    selected_device = df[df['oem_id'] == user_id_input]
    if not selected_device.empty:
        brand = selected_device['brand'].values[0]
        price = selected_device['price'].values[0]
        performance_score = selected_device['Performance_Score'].values[0]
        price_performance_ratio = selected_device['Price_Performance_Ratio'].values[0]
       
        print()
        print(f"For this {brand} brand and oem_id of {user_id_input} with a price of {price}: ")
        print(f"The Performance score of the device is {performance_score:.2f}")
        print(f"The higher the number (up to {df['Performance_Score'].max()}) the better the performance of the device.")
        print(f"The lower the number (down to {df['Performance_Score'].min():.2f}) the less the performance of the device.")
        print("---------------------------------------------------------------------------------------------------------------------")
        print(f"The Price Performance Ratio for the device is {price_performance_ratio:.2f}")
        print(f"The higher the number (up to {df['Price_Performance_Ratio'].max():.2f}) the better the performance relative to the price.")
        print(f"The closer it is to (down to {df['Price_Performance_Ratio'].min():.2f}), the less the performance of the device for its price.")
        
        # Add recommendations based on the price-performance ratio
        if price_performance_ratio >= 5:  # Good buy threshold is 5.0
            print("This is a good buy! It offers great performance for its price.")
        else:
            print("Based on the price performance ratio. \nPlease consider other options. It may not provide the best value for your money.")
    else:
        print(f"No device with oem_id {user_id_input} found in the dataset.")
        
price_performance_ratio()

In [ ]:
def get_cheapest_prices_by_brand():
    while True:
        try:
            brand_names = ", ".join(data_file['brand'].unique())
            print(f"Available device brand names in this dataset:\n{brand_names}")

            user_brand = input("Enter the brand to find the top 5 cheapest prices (or 'quit' to exit): ").strip().lower()

            if user_brand == 'quit':
                break  # Exit the loop if the user wants to quit

            filtered_data = data_file[data_file['brand'].str.lower() == user_brand]

            if not filtered_data.empty:
                top_5_cheapest = filtered_data.nsmallest(5, 'price')[['oem_id', 'price']]
                print(f"Top 5 cheapest prices for brand: {user_brand}")
                for _, row in top_5_cheapest.iterrows():
                    print(f"OEM ID: {row['oem_id']} - Price: {row['price']}")
            else:
                print(f"No data available for brand: {user_brand}")
        except Exception as e:
            print(f"An error occurred: {e}. Please try again.")

# Call the function to get top 5 cheapest prices by brand
get_cheapest_prices_by_brand()


In [ ]:
def average_mass_brand():
    try:
        # Group the data by 'manufacturer' and calculate the average 'weight_gram'
        manufact_avg_mass = data_file.groupby('manufacturer')['weight_gram'].mean().round(2).reset_index()

        if not manufact_avg_mass.empty:
            num_rows = 20
            start_pos = 0
            end_pos = min(start_pos + num_rows, len(manufact_avg_mass))

            while True:
                # Display the current set of rows
                print(manufact_avg_mass.iloc[start_pos:end_pos])

                if end_pos >= len(manufact_avg_mass):
                    print("No more rows available.")
                    break

                user_input = input("Enter 'next' to retrieve the next 20 rows, or 'quit' to exit: ").strip().lower()
                if user_input == 'next':
                    start_pos += num_rows
                    end_pos = min(start_pos + num_rows, len(manufact_avg_mass))
                elif user_input == 'quit':
                    break
        else:
            print("No data available for average mass by brand.")
    except Exception as e:
        print(f"An error occurred: {e}. Please try again.")

# Call the function to calculate average mass by brand
average_mass_brand()


In [ ]:
def get_brand_regions():
    while True:
        try:
            # Use groupby to count the occurrences of each band in each region
            brand_counts = data_file.groupby(['market_regions', 'brand']).size().reset_index(name='counts')

            # Now, you can find the top 5 regions for each band type
            top_regions_by_brand = brand_counts.groupby('brand').apply(lambda x: x.nlargest(5, 'counts'))

            # Drop the 'individual_bands' column to avoid conflicts when resetting the index
            top_regions_by_brand = top_regions_by_brand.drop(columns='brand')

            # Reset the index to avoid the ambiguity issue
            top_regions_by_brand = top_regions_by_brand.reset_index()
            
            brand_name = ", ".join(data_file['brand'].unique())
            print(f"The device brand names in this dataset are:\n {brand_name}")

            # Ask the user for the brand they want to see
            user_brand = input("Enter the brand you want to see: ").strip().lower()

            # Filter the results for the user-specified brand
            user_brand_data = top_regions_by_brand[top_regions_by_brand['brand'].str.lower() == user_brand]

            if not user_brand_data.empty:
                
                print(f"Top regions for brand: {user_brand}")
                for _, row in user_brand_data.iterrows():
                    print(f"Region: {row['market_regions']} Count: {row['counts']}")
                break  # Exit the loop if a valid brand is provided
            else:
                print(f"No data available for brand: {user_brand}")
        except Exception as e:
            print(f"An error occurred: {e}. Please try again.")

# Call the function to get top regions for a brand
get_brand_regions()


In [ ]:
# def get_top_regions_for_band():
bands = data_file['supported_cellular_bands'].tolist()  # Convert bands column to a list
bands_list = [band.strip() for sublist in [band.split(',') for band in bands] for band in sublist]  # Split each string by commas, flatten the list, and remove leading/trailing whitespace
band_counts = pd.Series(bands_list).value_counts().to_frame()  # Count the occurrence of each band
band_counts_df = band_counts.reset_index()

print(band_counts_df)
    

In [ ]:
# Assuming you have loaded the data into a DataFrame called 'loaded_data'
bands = data_file['supported_cellular_bands'].tolist()  # Convert amenities column to a list
bands_list = [band.strip() for sublist in [band.split(',') for band in bands] for band in sublist]  # Split each string by commas, flatten the list, and remove leading/trailing whitespace
band_counts = pd.Series(bands_list).value_counts().to_frame()  # Count the occurrence of each amenity
top_most_band = band_counts.index[0]  # Get the most popular amenity (first index of the value counts)
top_most_count = band_counts.iloc[0, 0]  # Get the count of the most popular amenity

print(f"The most popular amenity is '{top_most_band}' with a count of {top_most_count}.")

In [ ]:
# Get a list of unique cellular bands in the dataset
unique_bands = new_updated_data['individual_bands'].unique()

# Create a dictionary to store average prices for each band
average_prices = {}

# Iterate through each band and calculate the average price
for band in unique_bands:
    band_data = new_updated_data[new_updated_data['individual_bands'] == band]
    average_price = band_data['price'].mean()
    currency = band_data['price_currency'].iloc[0]
    average_prices[band] = (average_price, currency)

# Print the results
for band, (average_price, currency) in average_prices.items():
    print(f"Average price of devices within {band} is: {average_price:.2f} {currency}")

In [ ]:
# Group by year and month and calculate the average price for each month
monthly_average_prices_2020 = GBP_2020.groupby(['released_year', 'released_month'])['price'].mean().reset_index()
monthly_average_prices_2021 = GBP_2021.groupby(['released_year', 'released_month'])['price'].mean().reset_index()
monthly_average_prices_2022 = GBP_2022.groupby(['released_year', 'released_month'])['price'].mean().reset_index()
monthly_average_prices_2023 = GBP_2023.groupby(['released_year', 'released_month'])['price'].mean().reset_index()

monthly_average_prices_2020.rename(columns={'price': 'average_price(GBP)'}, inplace=True)
monthly_average_prices_2021.rename(columns={'price': 'average_price(GBP)'}, inplace=True)
monthly_average_prices_2022.rename(columns={'price': 'average_price(GBP)'}, inplace=True)
monthly_average_prices_2023.rename(columns={'price': 'average_price(GBP)'}, inplace=True)

# print(monthly_average_prices_2020.head())
# print(monthly_average_prices_2021.head())
# print(monthly_average_prices_2022.head())
# print(monthly_average_prices_2023.head())



# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8), sharey=True)

# Plot line graph for Year 2019
axs[0, 0].plot(monthly_average_prices_2020['average_price(GBP)'])
axs[0, 0].set_title('Year 2020')

# Plot line graph for Year 2020
axs[0, 1].plot(monthly_average_prices_2021['average_price(GBP)'])
axs[0, 1].set_title('Year 2021')

# Plot line graph for Year 2021
axs[1, 0].plot(monthly_average_prices_2022['average_price(GBP)'])
axs[1, 0].set_title('Year 2022')

# Plot line graph for Year 2022
axs[1, 1].plot(monthly_average_prices_2023['average_price(GBP)'])
axs[1, 1].set_title('Year 2023')

# Adjust spacing between subplots
fig.tight_layout()
fig.suptitle('Subplots of average price of devices for years 2020 to 2023 for GBP')
plt.subplots_adjust(top=0.9)
# Show the plot
plt.show()

In [3]:
import csv
# "device_features.csv"
file_location = input("Enter location: ")
# Assuming you have a list of lists called 'data'
with open(file_location, "r") as file:
    data = csv.reader(file)
    result_rows = []
    code_name = input("Enter the code name: ")

    # Create a flag to check if the code name is found
    code_name_found = False

    # Loop through each row in the CSV file
    for row in data:
        if row and row[23] == code_name:
            # If the code name is found, print the details
            value1 = row[41]
            value2 = row[2]
            value3 = row[23]
            value4 = row[44]
            value5 = row[45]
            row_dict = {
                "Band(WLAN)": value1,
                "Model name": value2,
                "RAM": value3,
                "Market regions": value4,
                "Date of addition": value5
            }
            result_rows.append(row_dict)

            code_name_found = True

    if not code_name_found:
        print("No match for the OEM ID details found.")   

    else:
        num_rows = 20
        start_pos = 0
        end_pos = min(start_pos + num_rows, len(result_rows))

        while True:
            for i in range(start_pos, end_pos):
                row = result_rows[i]
                print("Band(WLAN):", row["Band(WLAN)"])
                print("Model name:", row["Model name"])
                print("RAM:", row["RAM"])
                print("Market region:", row["Market regions"])
                print("Date of addition:", row["Date of addition"])
                print()

            if end_pos >= len(result_rows):
                print("No more rows available.")
                break

            user_input = input("Enter 'next' to retrieve the next 20 rows, or 'quit' to exit: ")
            if user_input == 'next':
                start_pos += num_rows
                end_pos = min(start_pos + num_rows, len(result_rows))
            elif user_input == 'quit':
                break


Enter location:  device_features.csv
Enter the code name:  3


Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U Galaxy A13 2022 Standard Edition TD-LTE US 32GB / SM-A135R4
RAM: 3
Market region: North America
Date of addition: 02-08-23 18:10

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U1/DS Galaxy A13 2022 Standard Edition Dual SIM TD-LTE US 32GB
RAM: 3
Market region: North America
Date of addition: 01-08-23 09:00

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U Galaxy A13 2022 Standard Edition TD-LTE US 32GB / SM-A135V
RAM: 3
Market region: North America
Date of addition: 31-07-23 23:26

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U Galaxy A13 2022 Standard Edition Dual SIM TD-LTE US 32GB / SM-A135T
RAM: 3
Market region: North America
Date of addition: 21-07-23 22:38

Band(WLAN): 802.11b,802.11g,802.11n
Model name: C110 2023 TD-LTE US 32GB N156DL
RAM: 3
Market region: North America
Date of addition: 29-06-23 14:25

Band(WLAN): 802.11b,802.11

Enter 'next' to retrieve the next 20 rows, or 'quit' to exit:  next


Band(WLAN): 802.11b,802.11g,802.11n
Model name: G11 2022 Global Dual SIM TD-LTE 32GB
RAM: 3
Market region: Asia,Eastern Europe,Europe,North America,Western Europe
Date of addition: 18-01-23 17:10

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: iPad 10.2-inch 2021 9th gen A2602 WiFi 64GB
RAM: 3
Market region: Africa,Asia,Australia,Central America,Eastern Europe,Europe,Middle East,North America,Oceania,South America,Southeast Asia,Worldwide
Date of addition: 25-11-21 14:36

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: iPad 10.2-inch 2021 9th gen A2602 WiFi 256GB
RAM: 3
Market region: Africa,Asia,Australia,Central America,Eastern Europe,Europe,Middle East,North America,Oceania,South America,Southeast Asia,Worldwide
Date of addition: 25-11-21 14:21

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: iPad 10.2-inch 2021 9th gen A2603 TD-LTE NA 64GB
RAM: 3
Market region: North America
Date of addition: 25-11-21 14:07

Band(WLAN): 802.11a,80

Enter 'next' to retrieve the next 20 rows, or 'quit' to exit:  quit


In [ ]:
import csv

# Initialize an empty set to store unique values
unique_brands = set()

# file_location = loaded_data
column_index = 9  # Adjust this index to match the column you want (0-based index)

try:
    # with open(file_location, 'r', encoding='utf-8') as file:
    #     reader = csv.reader(file)
    #     next(reader)  # Skip the header row if it exists
    for line in loaded_data:
        if column_index < len(line):
            unique_brands.add(line[column_index])

    # Convert the set to a list if needed
    unique_brands_list = list(unique_brands)

    for brand in unique_brands_list:
        print(brand)

except csv.Error:
    print(f"Invalid CSV file: {file_location}")


In [ ]:
# def get_oem_id_details(loaded_data):
#     while True:
#         try:
#             oem_id = input("Enter the OEM ID: ").strip()  # Trim leading/trailing whitespaces

#             # Create a flag to check if the OEM ID is found
#             oem_id_found = False

#             # Loop through each row in the loaded data
#             for row in loaded_data:
#                 if row and row[0].strip() == oem_id:  # Trim leading/trailing whitespaces in row[0]
#                     # If the OEM ID is found, print the details
#                     print(f"OEM ID: {oem_id}")
#                     print(f"Model: {row[2]}")
#                     print(f"Manufacturer: {row[6]}")
#                     print(f"Weight: {row[14]}")
#                     print(f"Price: {row[15]}")
#                     print(f"Price Unit: {row[16]}")

#                     oem_id_found = True
#                     break  # Exit the loop as we found the ID
#             break

#             if not oem_id_found:
#                 print("No match for the OEM ID details found.")
#         except ValueError:
#             print("Invalid input. Please enter an integer.")
# get_oem_id_details(loaded_data)

In [ ]:
# result_rows = []
# print("We have the following brand of phones:\n Samsung \n Xiaomi \n Sharp \n Sony \n Lenovo \n Asus \n BBK \n T-Mobile \n ZTE \n Nokia \n Microsoft \n LG \n Apple \n Motorol")
# brand_name = input("Enter the brand name: ")
# print("We have the following device types: \n Tablet \n Smartwatch \n Smartphone")
# device_type = input("Enter the device type():")   # add .lower or upper to ensure no error from entry
# min_price = input("The minimum price for your phone")# error handling(if they enter wrong stuffs or out of range of what we sell)
# max_price = input("The maximum price for your phone") # convert now or later to int or float


# # Create a flag to check if the code name is found
# brand_name_found = False

# # Loop through each row in the CSV file
# for row in loaded_data:
#     if row and row[23] == brand_name:
#         if row and row[9] == device_type:
#             if row and row[15] >= min_price and row[15] <= max_price: 
#                 # If the brand name,device type and price range is found, print the details
#                 value1 = row[42]
#                 value2 = row[24]
#                 value3 = row[19]
#                 value4 = row[29]
#                 value5 = row[35]
#                 row_dict = {
#                     "Battery Capacity": value1,
#                     "Storage Space": value2,
#                     "Additional Software Details": value3,
#                     "Pixel Density": value4,
#                     "Sim-Card-Type": value5
#                 }
#                 result_rows.append(row_dict)

#                 code_name_found = True

# if not brand_name_found:
#     print("No match for the brand name details found.")   

# else:
#     num_rows = 20
#     start_pos = 0
#     end_pos = min(start_pos + num_rows, len(result_rows))

#     while True:
#         for i in range(start_pos, end_pos):
#             row = result_rows[i]
#             print("Battery Capacity: ", row["Battery Capacity"])
#             print("Storage Space: ", row["Storage Space"])
#             print("Additional Software Details: ", row["Additional Software Details"])
#             print("Pixel Density: ", row["Pixel Density"])
#             print("Sim-Card-Type: ", row["Sim-Card-Type"])
#             print()

#         if end_pos >= len(result_rows):
#             print("No more rows available.")
#             break

#         user_input = input("Enter 'next' to retrieve the next 20 rows, or 'quit' to exit: ")
#         if user_input == 'next':
#             start_pos += num_rows
#             end_pos = min(start_pos + num_rows, len(result_rows))
#         elif user_input == 'quit':
#             break


In [ ]:
# "device_features.csv"
file_location = input("Enter location: ")
with open(file_location, "r") as file:
    csv_reader = csv.reader(file)
    # for row in csv_reader:
        # print(row)
# a1. Retrieve the model name, manufacturer, weight, price, and price unit for the device(s) based on the oem_id.
# Loop through each row in the CSV file
result_rows = []
while True:
    oem_id = input("Enter the oem id: ")
    for row in file:
        # Convert row[0] to integer for comparison
        if row[0] == oem_id:
            value1 = row[2]
            value2 = row[6]
            value3 = row[14]
            value4 = row[15]
            value5 = row[16]
            # If it is, add the entire row to the list of matching rows
            result_rows.append(row)
            # Print the matching rows
    if result_rows:
        print("The Oem Id number {} details are :\n".format(oem_id))
        print("The model of the device is :", value1)
        print("The manufactuerer is :", value2)
        print("The weight of the device is :", value3)
        print("The price of the device is : ", value4)
        print("The price unit is : ", value5)
    else:
        print("No match for the oem Id details found.")
return